# Models

Neural networks in tinygrad are really just represented by the operations performed on tensors.
These operations are commonly grouped into the `__call__` method of a class which allows modularization and reuse of these groups of operations.
These classes do not need to inherit from any base class, in fact if they don't need any trainable parameters they don't even need to be a class!

In [1]:
import numpy as np
from tinygrad.helpers import Timing
from tinygrad.tensor import Tensor
from tinygrad.helpers import dtypes

An example of this would be the `nn.Linear` class which represents a linear layer in a neural network.

In [2]:
class Linear:
  def __init__(self, in_features, out_features, bias=True, initialization: str='kaiming_uniform'):
    self.weight = getattr(Tensor, initialization)(out_features, in_features)
    self.bias = Tensor.zeros(out_features) if bias else None

  def __call__(self, x):
    return x.linear(self.weight.transpose(), self.bias)

There are more neural network modules already implemented in [nn](/tinygrad/nn/__init__.py), and you can also implement your own.

We will be implementing a simple neural network that can classify handwritten digits from the MNIST dataset.
Our classifier will be a simple 2 layer neural network with a Leaky ReLU activation function.
It will use a hidden layer size of 128 and an output layer size of 10 (one for each digit) with no bias on either Linear layer.

In [3]:
class TinyNet:
  def __init__(self):
    self.l1 = Linear(784, 128, bias=False)
    self.l2 = Linear(128, 10, bias=False)

  def __call__(self, x):
    x = self.l1(x)
    x = x.leakyrelu()
    x = self.l2(x)
    return x

net = TinyNet()

We can see that the forward pass of our neural network is just the sequence of operations performed on the input tensor `x`.
We can also see that functional operations like `leakyrelu` are not defined as classes and instead are just methods we can just call.
Finally, we just initialize an instance of our neural network, and we are ready to start training it.

## Training

Now that we have our neural network defined we can start training it.
Training neural networks in tinygrad is super simple.
All we need to do is define our neural network, define our loss function, and then call `.backward()` on the loss function to compute the gradients.
They can then be used to update the parameters of our neural network using one of the many optimizers in [optim.py](/tinygrad/nn/optim.py).

For our loss function we will be using sparse categorical cross entropy loss. The implementation below is taken from [tensor.py](../../tinygrad/tensor.py), it's copied below to highlight an important detail of tinygrad.

In [4]:
def sparse_categorical_crossentropy(self, Y, ignore_index=-1) -> Tensor:
    loss_mask = Y != ignore_index
    y_counter = Tensor.arange(self.shape[-1], dtype=dtypes.int32, requires_grad=False, device=self.device).unsqueeze(0).expand(Y.numel(), self.shape[-1])
    y = ((y_counter == Y.flatten().reshape(-1, 1)).where(-1.0, 0) * loss_mask.reshape(-1, 1)).reshape(*Y.shape, self.shape[-1])
    return self.log_softmax().mul(y).sum() / loss_mask.sum()

As we can see in this implementation of cross entropy loss, there are certain operations that tinygrad does not support natively.
Namely, operations that are load/store or assigning a value to a tensor at a certain index.
Load/store ops are not supported in tinygrad natively because they add complexity when trying to port to different backends, 90% of the models out there don't use/need them, and they can be implemented like it's done above with an `arange` mask.

For our optimizer we will be using the traditional stochastic gradient descent optimizer with a learning rate of 3e-4.

In [5]:
from tinygrad.nn.optim import SGD

opt = SGD([net.l1.weight, net.l2.weight], lr=3e-4)

We can see that we are passing in the parameters of our neural network to the optimizer.
This is due to the fact that the optimizer needs to know which parameters to update.
There is a simpler way to do this just by using `get_parameters(net)` from `tinygrad.nn.state` which will return a list of all the parameters in the neural network.
The parameters are just listed out explicitly here for clarity.

Now that we have our network, loss function, and optimizer defined all we are missing is the data to train on!
There are a couple of dataset loaders in tinygrad located in [/extra/datasets](/extra/datasets).
We will be using the MNIST dataset loader.

In [6]:
import os, gzip, tarfile, pickle
from tinygrad.helpers import fetch

def fetch_mnist(tensors=False):
  parse = lambda file: np.frombuffer(gzip.open(file).read(), dtype=np.uint8).copy()
  BASE_URL = "https://storage.googleapis.com/cvdf-datasets/mnist/"   # http://yann.lecun.com/exdb/mnist/ lacks https
  X_train = parse(fetch(f"{BASE_URL}train-images-idx3-ubyte.gz"))[0x10:].reshape((-1, 28*28)).astype(np.float32)
  Y_train = parse(fetch(f"{BASE_URL}train-labels-idx1-ubyte.gz"))[8:]
  X_test = parse(fetch(f"{BASE_URL}t10k-images-idx3-ubyte.gz"))[0x10:].reshape((-1, 28*28)).astype(np.float32)
  Y_test = parse(fetch(f"{BASE_URL}t10k-labels-idx1-ubyte.gz"))[8:]
  if tensors: return Tensor(X_train).reshape(-1, 1, 28, 28), Tensor(Y_train), Tensor(X_test).reshape(-1, 1, 28, 28), Tensor(Y_test)
  else: return X_train, Y_train, X_test, Y_test

Now we have everything we need to start training our neural network.
We will be training for 1000 steps with a batch size of 64.

We use `with Tensor.train()` set the internal flag `Tensor.training` to `True` during training.
Upon exit, the flag is restored to its previous value by the context manager.

In [7]:
X_train, Y_train, X_test, Y_test = fetch_mnist()

with Tensor.train():
  for step in range(1000):
    # Random sample a batch
    samp = np.random.randint(0, X_train.shape[0], size=(64))
    batch = Tensor(X_train[samp], requires_grad=False)
    # Get the corresponding labels
    labels = Tensor(Y_train[samp])

    # Forward pass
    out = net(batch)

    # Compute loss
    loss = sparse_categorical_crossentropy(out, labels)

    # Zero gradients
    opt.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    opt.step()

    # Calculate accuracy
    pred = out.argmax(axis=-1)
    acc = (pred == labels).mean()

    if step % 100 == 0:
      print(f"Step {step+1} | Loss: {loss.numpy()} | Accuracy: {acc.numpy()}")

Step 1 | Loss: 162.09005737304688 | Accuracy: 0.078125
Step 101 | Loss: 6.010622978210449 | Accuracy: 0.796875
Step 201 | Loss: 4.344995498657227 | Accuracy: 0.84375
Step 301 | Loss: 5.003681659698486 | Accuracy: 0.875
Step 401 | Loss: 2.147794246673584 | Accuracy: 0.890625
Step 501 | Loss: 3.499648094177246 | Accuracy: 0.84375
Step 601 | Loss: 3.2069578170776367 | Accuracy: 0.921875
Step 701 | Loss: 2.139812707901001 | Accuracy: 0.921875
Step 801 | Loss: 5.188339710235596 | Accuracy: 0.8125
Step 901 | Loss: 0.40168359875679016 | Accuracy: 0.9375


## Evaluation

Now that we have trained our neural network we can evaluate it on the test set.
We will be using the same batch size of 64 and will be evaluating for 1000 of those batches.

In [8]:
with Timing("Time: "):
  avg_acc = 0
  for step in range(1000):
    # Random sample a batch
    samp = np.random.randint(0, X_test.shape[0], size=(64))
    batch = Tensor(X_test[samp], requires_grad=False)

    # Get the corresponding labels
    labels = Y_test[samp]

    # Forward pass
    out = net(batch)

    # Calculate the accuracy
    pred = out.argmax(axis=-1).numpy()
    avg_acc += (pred == labels).mean()
  print(f"Test Accuracy: {avg_acc / 1000}")

Test Accuracy: 0.894109375
Time: 4893.06 ms


## Saving and Loading Models

The standard weight format for tinygrad is [safetensors](https://github.com/huggingface/safetensors). This means that you can load the weights of any model also using safetensors into tinygrad.
There are functions in [state.py](/tinygrad/nn/state.py) to save and load models to and from this format.

In [9]:
from tinygrad.nn.state import safe_save, safe_load, get_state_dict, load_state_dict
import pathlib
import shutil

# We need to make sure the directory exists
pathlib.Path("./tmp/models").mkdir(parents=True, exist_ok=True)

# First we need the state dict of our model
state_dict = get_state_dict(net)

# Then we can just save it to a file
safe_save(state_dict, "./tmp/models/model_tinynet.safetensors")

# And load it back in
state_dict = safe_load("./tmp/models/model_tinynet.safetensors")
load_state_dict(net, state_dict)

# Remove the file (example purposes)
shutil.rmtree("./tmp")

ram used:  0.00 GB, l2.weight                                         : 100%|██████████| 2/2 [00:00<00:00, 50.97it/s]

loaded weights in 138.58 ms, 0.00 GB loaded at 0.01 GB/s


Many of the models in the [models/](/models) folder have a `load_from_pretrained` method that will download and load the weights for you. These usually are pytorch weights meaning that you would need pytorch installed to load them.

## Visualizing the Computation Graph

It is possible to visualize the computation graph of a neural network using [graphviz](https://graphviz.org/).

This is easily done by running a single pass (forward or backward!) of the neural network with the environment variable `GRAPH` set to `1`.
The graph will be saved to `/tmp/net.svg` by default.

## Recommendation

And that's it!

We highly recommend you check out the [examples/](/examples) folder for more examples of using tinygrad.
Reading the source code of tinygrad is also a great way to learn how it works.
Specifically the tests in [test/](/test) are a great place to see how to use and the semantics of the different operations.
There are also a bunch of models implemented in [models/](/models) that you can use as a reference.

Additionally, feel free to ask questions in the `#learn-tinygrad` channel on the [Discord](https://discord.gg/beYbxwxVdx). Don't ask to ask, just ask!